In [1]:
import gensim
from gensim.parsing.preprocessing import stem_text
from gensim.utils import simple_preprocess
import numpy as np
import nltk 
import pandas as pd
np.random.seed(2018)

In [2]:
import os,glob
import string
import array
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
wordnet_lemmatizer = WordNetLemmatizer()
train_docs=[]
test_docs=[]
def clean(text):
        tokens = nltk.word_tokenize(text)
        tokens = [w.lower() for w in tokens]
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        words = [word for word in stripped if word.isalpha()]
        stop_words = stopwords.words('english')
        stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])
        words = [w for w in words if not w in stop_words]
        words = list(map(stem_text, words))
        return words
    
def create_docs(path,documents):
    for filename in glob.glob(os.path.join(path, '*.txt')):
       with open(os.path.join(os.getcwd(), filename), 'r') as f: # open in readonly mode
          # do your stuff
            text = f.read()
            documents.append(clean(text))
            f.close()

path = '/home/xmliszt/Documents/git/capstone-topic-modelling/BBC News Summary/Summaries/business'
create_docs(path,train_docs)
path = '/home/xmliszt/Documents/git/capstone-topic-modelling/BBC News Summary/Summaries/entertainment'
create_docs(path,train_docs)
path = '/home/xmliszt/Documents/git/capstone-topic-modelling/BBC News Summary/Summaries/politics'
create_docs(path,train_docs)
path = '/home/xmliszt/Documents/git/capstone-topic-modelling/BBC News Summary/Summaries/sport'
create_docs(path,train_docs)
path = '/home/xmliszt/Documents/git/capstone-topic-modelling/BBC News Summary/Summaries/tech'
create_docs(path,train_docs)

[nltk_data] Downloading package punkt to /home/xmliszt/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/xmliszt/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
train_docs[1:5]

[['howev',
  'slowerthanexpect',
  'fourth',
  'quarter',
  'point',
  'modest',
  'growth',
  'tradedriven',
  'economi',
  'global',
  'technolog',
  'demand',
  'fall',
  'backth',
  'economi',
  'poor',
  'perform',
  'juli',
  'septemb',
  'period',
  'follow',
  'four',
  'consecut',
  'quarter',
  'doubledigit',
  'growth',
  'singapor',
  'bounc',
  'back',
  'strongli',
  'effect',
  'deadli',
  'sar',
  'viru',
  'advanc',
  'secondfastest',
  'asia',
  'china',
  'led',
  'growth',
  'kei',
  'manufactur',
  'sector',
  'surpris',
  'weak',
  'fourth',
  'quarter',
  'numberthat',
  'third',
  'quarter',
  'fell',
  'well',
  'analyst',
  'forecast'],
 ['prosecut',
  'star',
  'wit',
  'former',
  'worldcom',
  'financi',
  'chief',
  'scott',
  'sullivan',
  'said',
  'mr',
  'ebber',
  'order',
  'account',
  'adjust',
  'firm',
  'tell',
  'hit',
  'book',
  'howev',
  'ms',
  'cooper',
  'said',
  'mr',
  'sullivan',
  'mention',
  'anyth',
  'uncomfort',
  'worldcom',
 

Bag of words

In [4]:
dictionary = gensim.corpora.Dictionary(train_docs)
# for k, v in dictionary.iteritems():
#     print(k, v)
#     count += 1
#     if count > 10:
#         break
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
#For each document we create a dictionary reporting how many words and how many times those words appear
bow_corpus = [dictionary.doc2bow(doc) for doc in train_docs]
bow_doc_10 = bow_corpus[10]
for i in range(len(bow_doc_10)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_10[i][0], 
                                               dictionary[bow_doc_10[i][0]], 
bow_doc_10[i][1]))

Word 3 ("export") appears 1 time.
Word 11 ("kept") appears 1 time.
Word 15 ("oil") appears 1 time.
Word 18 ("price") appears 1 time.
Word 23 ("strong") appears 1 time.
Word 31 ("china") appears 1 time.
Word 35 ("fall") appears 1 time.
Word 41 ("growth") appears 1 time.
Word 48 ("point") appears 1 time.
Word 60 ("account") appears 2 time.
Word 68 ("chief") appears 1 time.
Word 75 ("financ") appears 2 time.
Word 82 ("hit") appears 1 time.
Word 85 ("meet") appears 1 time.
Word 87 ("mr") appears 2 time.
Word 113 ("deficit") appears 2 time.
Word 116 ("fund") appears 1 time.
Word 133 ("turn") appears 1 time.
Word 135 ("us") appears 6 time.
Word 170 ("sinc") appears 1 time.
Word 174 ("trade") appears 1 time.
Word 196 ("mondai") appears 1 time.
Word 209 ("share") appears 1 time.
Word 217 ("ad") appears 1 time.
Word 237 ("japan") appears 2 time.
Word 246 ("seen") appears 1 time.
Word 251 ("world") appears 1 time.
Word 278 ("trend") appears 1 time.
Word 291 ("much") appears 1 time.
Word 318 ("af

TF-IDF

In [5]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.11104392088587417),
 (1, 0.12409600975358015),
 (2, 0.11021474446002884),
 (3, 0.1681205051403281),
 (4, 0.1440329094089247),
 (5, 0.11903882545112768),
 (6, 0.17421638093068081),
 (7, 0.11672755837914486),
 (8, 0.184649872323546),
 (9, 0.16485744925951995),
 (10, 0.1316827363354427),
 (11, 0.16485744925951995),
 (12, 0.105565434141836),
 (13, 0.47981962852214355),
 (14, 0.12042087087136809),
 (15, 0.1440329094089247),
 (16, 0.1863848761002872),
 (17, 0.19009347123018325),
 (18, 0.12007094223769688),
 (19, 0.3596965555914307),
 (20, 0.17166764882943514),
 (21, 0.11132410367616014),
 (22, 0.12856247021713252),
 (23, 0.12566414291044306),
 (24, 0.23672996945805594),
 (25, 0.16382533247295072),
 (26, 0.31110431528565613),
 (27, 0.04509917690726134)]


Run LDA

In [6]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.004*"mr" + 0.004*"us" + 0.003*"year" + 0.003*"economi" + 0.003*"tax" + 0.003*"bank" + 0.003*"peopl" + 0.003*"growth" + 0.003*"govern" + 0.003*"price"
Topic: 1 Word: 0.004*"win" + 0.004*"player" + 0.004*"game" + 0.003*"mr" + 0.003*"plai" + 0.003*"england" + 0.003*"two" + 0.003*"six" + 0.003*"number" + 0.003*"rugbi"
Topic: 2 Word: 0.007*"film" + 0.004*"best" + 0.004*"firm" + 0.003*"mr" + 0.003*"new" + 0.003*"us" + 0.003*"sale" + 0.003*"year" + 0.002*"director" + 0.002*"compani"
Topic: 3 Word: 0.006*"mobil" + 0.005*"music" + 0.004*"us" + 0.004*"phone" + 0.003*"sale" + 0.003*"show" + 0.003*"peopl" + 0.003*"new" + 0.003*"digit" + 0.003*"network"
Topic: 4 Word: 0.006*"film" + 0.006*"mr" + 0.005*"game" + 0.004*"blair" + 0.004*"lord" + 0.003*"plai" + 0.003*"brown" + 0.003*"labour" + 0.003*"howard" + 0.003*"first"


Classify docs in the model

In [7]:
score_list=[]
for index, score in sorted(lda_model_tfidf[bow_corpus[500]], key=lambda tup: -1*tup[1]):
    score_list.append(score)
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))
for i in range(len(score_list)):
    if score_list[i] == max(score_list):
        maxIndex = i
    else:
        pass
print(score_list)
print(maxIndex)


Score: 0.9829461574554443	 
Topic: 0.004*"mr" + 0.004*"us" + 0.003*"year" + 0.003*"economi" + 0.003*"tax" + 0.003*"bank" + 0.003*"peopl" + 0.003*"growth" + 0.003*"govern" + 0.003*"price"
[0.98294616]
0


Test on unseen

In [8]:
# unseen_document = 'How a Pentagon deal became an identity crisis for Google'
unseen_document = 'While the idea was popular amongst the children, it initially received some resistance from the teachers and there were problems with laptops getting broken. However, Mr Negroponte has adapted the idea to his own work in Cambodia where he set up two schools together with his wife and gave the children laptops. "We put in 25 laptops three years ago , only one has been broken, the kids cherish these things, its also a TV a telephone and a games machine, not just a textbook." Mr Negroponte wants the laptops to become more common than mobile phones but conceded this was ambitious. Nokia make 200 million cell phones a year, so for us to claim were going to make 200 million laptops is a big number, but were not talking about doing it in three or five years, were talking about months. He plans to be distributing them by the end of 2006 and is already in discussion with the Chinese education ministry who are expected to make a large order. In China they spend 17 per child per year on textbooks. Thats for five or six years, so if we can distribute and sell laptops in quantities of one million or more to ministries of education that cheaper'
bow_vector = dictionary.doc2bow(clean(unseen_document))

for index, score in sorted(lda_model_tfidf[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model_tfidf.print_topic(index, 7)))

Score: 0.5156767964363098	 Topic: 0.006*"mobil" + 0.005*"music" + 0.004*"us" + 0.004*"phone" + 0.003*"sale" + 0.003*"show" + 0.003*"peopl"
Score: 0.375169962644577	 Topic: 0.004*"mr" + 0.004*"us" + 0.003*"year" + 0.003*"economi" + 0.003*"tax" + 0.003*"bank" + 0.003*"peopl"
Score: 0.10413332283496857	 Topic: 0.006*"film" + 0.006*"mr" + 0.005*"game" + 0.004*"blair" + 0.004*"lord" + 0.003*"plai" + 0.003*"brown"


Word Cloud

In [9]:
# # 1. Wordcloud of Top N words in each topic
# from matplotlib import pyplot as plt
# from wordcloud import WordCloud, STOPWORDS
# from nltk.corpus import stopwords
# import matplotlib.colors as mcolors

# cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]  # more colors: 'mcolors.XKCD_COLORS'

# cloud = WordCloud(stopwords = set(stopwords.words('english')),
#                   background_color='white',
#                   width=2500,
#                   height=1800,
#                   max_words=20,
#                   colormap='tab10',
#                   color_func=lambda *args, **kwargs: cols[i],
#                   prefer_horizontal=1.0)

# topics = lda_model_tfidf.show_topics(formatted=False)

# fig, axes = plt.subplots(1, 5, figsize=(10,10), sharex=True, sharey=True)

# for i, ax in enumerate(axes.flatten()):
#     fig.add_subplot(ax)
#     topic_words = dict(topics[i][1])
#     cloud.generate_from_frequencies(topic_words, max_font_size=300)
#     plt.gca().imshow(cloud)
#     plt.gca().set_title('Topic ' + str(i), fontdict=dict(size=16))
#     plt.gca().axis('off')


# plt.subplots_adjust(wspace=0, hspace=0)
# plt.axis('off')
# plt.margins(x=0, y=0)
# plt.tight_layout()
# plt.show()

t-SNE

In [16]:
# Get topic weights and dominant topics ------------
from sklearn.manifold import TSNE
from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook

# Get topic weights
topic_weights = []
for i, row_list in enumerate(lda_model_tfidf[bow_corpus]):
    row_list=sorted(row_list, key=lambda tup: -1*tup[1])
    topic_weights.append(row_list[0][1])
# print(topic_weights)

# Array of topic weights    
arr = pd.DataFrame(topic_weights).fillna(0).values

# Keep the well separated points (optional)
arr = arr[np.amax(arr, axis=1) > 0.35]
print(arr.shape)

# Dominant topic number in each doc
topic_num = np.argmax(arr, axis=1)

# tSNE Dimension Reduction
tsne_model = TSNE(n_components=2, verbose=1, random_state=0, angle=.99, init='pca')
tsne_lda = tsne_model.fit_transform(arr)

# # Plot the Topic Clusters using Bokeh
import matplotlib.colors as mcolors
output_notebook()
n_topics = 4
mycolors = np.array([color for name, color in mcolors.TABLEAU_COLORS.items()])
plot = figure(title="t-SNE Clustering of {} LDA Topics".format(n_topics), 
              plot_width=900, plot_height=700)
plot.scatter(x=tsne_lda[:,0], y=tsne_lda[:,1], color=mycolors[topic_num])
show(plot)

(2220, 1)
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 2220 samples in 0.001s...
[t-SNE] Computed neighbors for 2220 samples in 0.020s...
[t-SNE] Computed conditional probabilities for sample 1000 / 2220
[t-SNE] Computed conditional probabilities for sample 2000 / 2220
[t-SNE] Computed conditional probabilities for sample 2220 / 2220
[t-SNE] Mean sigma: 0.000358


ValueError: n_components=2 must be between 1 and min(n_samples, n_features)=1 with svd_solver='randomized'